In [2]:
'''
В модели генитор (Genitor) используется специфичный способ отбора.
Вначале, как и полагается, популяция инициализируется, и ее особи оцениваются.
Затем выбираются случайным образом две особи, скрещиваются, причем получается только один потомок,
который оценивается и занимает место менее приспособленной особи в популяции (а не одного из родителей!).
После этого снова случайным образом выбираются две особи, и их потомок занимает место
родительской особи с самой низкой приспособленностью. Таким образом, на каждом шаге в популяции
обновляется лишь одна особь. Процесс продолжается до тех пор, пока пригодности хромосом не станут одинаковыми. 
В данный алгоритм можно добавить мутацию потомка после его создания. 
Критерий окончания процесса, как и вид кроссинговера и мутации, можно выбирать разными способами.''';

In [261]:
# МЫ МАКСИМИЗИРУЕМ

In [288]:
from random import sample
import numpy as np

In [289]:
# num of symbols after point
PRECISION = 8
BITS = 64
BITS_CODE = '0{0}b'.format(BITS)

In [343]:
def square_func(vector):
    return -np.sum(vector ** 2)

def rastrigin_func(vector):
    return np.sum(vector**2 - 10 * np.cos(2*np.pi*vector))

In [347]:
FUNCTION = square_func

In [348]:
POPULATION_LENGTH = 100
population = generate_population(POPULATION_LENGTH, 2, range_=6.)
#print(list(map(lambda x:x.features, population)))

for _ in range(1000):
    sp1, sp2 = sample(population, 2)
    child = sp1.cross_over(sp2).mutate()
    population.sort(key=lambda x: -x.assess(FUNCTION))
    if population[POPULATION_LENGTH - 1].assess(FUNCTION) < child.assess(FUNCTION):
        population[POPULATION_LENGTH - 1] = child

    idsp1, idsp2 = sample(range(0, POPULATION_LENGTH), 2)
    sp1 = population[idsp1]
    sp2 = population[idsp2]
    child = sp1.cross_over(sp2).mutate()
    if sp1.assess(FUNCTION) < sp2.assess(FUNCTION):
        population[idsp1] = child
    else:
        population[idsp2] = child
print(list(map(lambda x:x.features, population)))
print(list(map(lambda x:x.assess(FUNCTION), population)))

[array([-0.00057728, -0.04194952]), array([-0.00058261, -0.04194992]), array([-0.0002651 , -0.04196375]), array([-0.00030399, -0.04196369]), array([-0.00026094, -0.04196406]), array([-0.00030438, -0.04196968]), array([-0.00076791, -0.04197048]), array([-0.00011881, -0.0419857 ]), array([-0.00110907, -0.04198576]), array([-0.00119722, -0.04198562]), array([-0.00119725, -0.04198566]), array([-0.00091533, -0.04199279]), array([-0.00095629, -0.04199295]), array([-0.00105264, -0.0419916 ]), array([-0.00107661, -0.04199215]), array([-0.00094734, -0.04200727]), array([-0.00122766, -0.04200471]), array([-0.00059278, -0.04203426]), array([-0.00032678, -0.0420516 ]), array([-0.00232518, -0.04198947]), array([-0.00025999, -0.04205501]), array([-0.00250793, -0.04198578]), array([-0.00028934, -0.04207009]), array([-0.00029442, -0.04207404]), array([-0.0002625, -0.042075 ]), array([-0.00026506, -0.04207532]), array([-0.00091589, -0.04206741]), array([-0.00012571, -0.04207984]), array([-0.00110887, -

In [297]:
def generate_population(num_species, n_dim, range_= 50.):
    population = []
    for _ in range(num_species):
        population.append(Species(n_dim, range_))
    return population

In [264]:
def invert(bit):
    return '0' if bit == '1' else '1'

In [265]:
def generate_vector(n_dim, range_):
    return np.random.randn(n_dim) * range_ * 0.5

In [266]:
def decimal2binary(vector):
    return list(map(lambda x: format(int(x * 10 ** PRECISION), BITS_CODE), list(vector)))

In [267]:
def binary2decimal(vector):
    return np.array(list(map(lambda x: int(x, 2) / (10 ** PRECISION), vector)))

In [339]:
class Species:
    def __init__(self, n_dim=1, range_=50., features=[]):
        if len(features) == 0:
            self.features = generate_vector(n_dim, range_)
        else:
            self.features = features
    
    def mutate(self, rate=1):
        genetic_code = decimal2binary(self.features)
        for i, feature in enumerate(genetic_code):
            chromosome = np.random.randint(BITS*2//3, BITS-1)
            if feature[0] == '-':
                chromosome += 1
            bit = feature[chromosome]
            mutated = feature[:chromosome] + invert(bit) + feature[chromosome + 1:]
            genetic_code[i] = mutated
        self.features = binary2decimal(genetic_code)
        return self
        
    def cross_over(self, partner):
        gens1 = decimal2binary(self.features)
        gens2 = decimal2binary(partner.features)
        child_gens = []
        for i, feature in enumerate(gens1):
            chromosome = np.random.randint(BITS//2, BITS)
            if feature[0] == '-':
                chromosome += 1
            child_gens.append(feature[:chromosome] + gens2[i][chromosome:])
        return Species(features = binary2decimal(child_gens))
    
    def assess(self, function):
        return function(self.features)

In [269]:
a = Species(2, 10.)
b = Species(2, 10.)

In [281]:
print(a.features)
print(b.features)

[-4.64577979  0.0955415 ]
[-3.16795867 -3.24623143]


In [282]:
a.mutate()
a.features

array([-4.64577947,  0.09554406])

In [283]:
c = a.cross_over(b)

In [284]:
print(c.features)

[-4.64645082  0.09526055]


In [285]:
vec = generate_vector(2, 50.)
print(vec)

[-6.73293848  2.30164937]


In [286]:
vec_bin = decimal2binary(vec)
print(vec_bin)

['-000000000000000000000000000000000101000001000011010011000011000', '0000000000000000000000000000000000001101101110000000100111001000']


In [287]:
binary2decimal(vec_bin)

array([-6.73293848,  2.30164936])